<a href="https://colab.research.google.com/github/klebster2/gwern-poetry-generation/blob/main/GWERN_GPT2_lyric_finetuning.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
# You can skip this but it helps to save weights/data in drive
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [3]:
!nvidia-smi
# run this to check the type of GPU. You want to switch to GPU for chatting with the model.

NVIDIA-SMI has failed because it couldn't communicate with the NVIDIA driver. Make sure that the latest NVIDIA driver is installed and running.



In [4]:
!git clone https://github.com/nshepperd/gpt-2.git ./nsheppy-gpt-2

fatal: destination path './nsheppy-gpt-2' already exists and is not an empty directory.


In [5]:
import os
os.chdir('./nsheppy-gpt-2')

In [6]:
# Download model weights
!python download_model.py 117M

Fetching checkpoint: 1.00kit [00:00, 668kit/s]                                                      
Fetching encoder.json: 1.04Mit [00:00, 4.97Mit/s]                                                   
Fetching hparams.json: 1.00kit [00:00, 728kit/s]                                                    
Fetching model.ckpt.data-00000-of-00001: 498Mit [00:09, 50.7Mit/s]                                  
Fetching model.ckpt.index: 6.00kit [00:00, 3.65Mit/s]                                               
Fetching model.ckpt.meta: 472kit [00:00, 2.94Mit/s]                                                 
Fetching vocab.bpe: 457kit [00:00, 2.85Mit/s]                                                       


In [7]:
!pip install lexical-diversity

### Check downloads were successful by checking encoder.json only.

In [8]:
!du -h models/117M/*

4.0K	models/117M/checkpoint
1020K	models/117M/encoder.json
4.0K	models/117M/hparams.json
475M	models/117M/model.ckpt.data-00000-of-00001
8.0K	models/117M/model.ckpt.index
464K	models/117M/model.ckpt.meta
448K	models/117M/vocab.bpe


In [9]:
!jq -h 2>/dev/null || apt-get install jq

jq - commandline JSON processor [version 1.5-1-a5b5cbe]
Usage: jq [options] <jq filter> [file...]

	jq is a tool for processing JSON inputs, applying the
	given filter to its JSON text inputs and producing the
	filter's results as JSON on standard output.
	The simplest filter is ., which is the identity filter,
	copying jq's input to its output unmodified (except for
	formatting).
	For more advanced filters see the jq(1) manpage ("man jq")
	and/or https://stedolan.github.io/jq

	Some of the options include:
	 -c		compact instead of pretty-printed output;
	 -n		use `null` as the single input value;
	 -e		set the exit status code based on the output;
	 -s		read (slurp) all inputs into an array; apply filter to it;
	 -r		output raw strings, not JSON texts;
	 -R		read raw strings, not JSON texts;
	 -C		colorize JSON;
	 -M		monochrome (don't colorize JSON);
	 -S		sort keys of objects on output;
	 --tab	use tabs for indentation;
	 --arg a v	set variable $a to value <v>;
	 --argjson a v	set v

Commands copied from [GWERN - 'Training GPT-2-117M To Generate Poetry'](https://www.gwern.net/GPT-2#training-gpt-2-117m-to-generate-poetry)

In [10]:
%%bash
## DATA DOWNLOAD
wget 'http://static.decontextualize.com/gutenberg-poetry-v001.ndjson.gz'
gunzip gutenberg-poetry-v001.ndjson.gz
cat gutenberg-poetry-v001.ndjson | jq .s | sed -e 's/^.//' -e 's/.$//' -e 's/\\//g' \
    >> gutenberg-poetry-v001.txt ## delete JSON quoting
shuf gutenberg-poetry-v001.txt | head ## random poetry lines:

# e.g.
# For the black bat, night, has flown,
# The other's fate, Gaville, still dost rue.
# That make me mad. Oh, save me from those eyes!

du -h gutenberg-poetry-v001.txt; wc gutenberg-poetry-v001.txt
# 117M  gutenberg-poetry-v001.txt
# 3085117  21959786 121730091 gutenberg-poetry-v001.txt

quique ortas proauo fregit Achille domos,
All nations have believed that from the dead
I would rather write prose--
Less devilish-cunning, franklier Hebrew-viced,
So doth our Charles move also; while he posts
They vow'd to ask the Gods.
Whithaht a stypher i’ the world—
Never such an ambuscade
And many a stubborn Badenoch-man,
Ode (Thou youngest virgin-daughter of the skies)
233M	gutenberg-poetry-v001.txt
  6170234  43919572 243460182 gutenberg-poetry-v001.txt


--2022-05-13 19:13:52--  http://static.decontextualize.com/gutenberg-poetry-v001.ndjson.gz
Resolving static.decontextualize.com (static.decontextualize.com)... 207.244.116.232
Connecting to static.decontextualize.com (static.decontextualize.com)|207.244.116.232|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 54837372 (52M) [application/x-gzip]
Saving to: ‘gutenberg-poetry-v001.ndjson.gz’

     0K .......... .......... .......... .......... ..........  0%  976K 55s
    50K .......... .......... .......... .......... ..........  0% 1.90M 41s
   100K .......... .......... .......... .......... ..........  0% 94.7M 28s
   150K .......... .......... .......... .......... ..........  0% 1.93M 27s
   200K .......... .......... .......... .......... ..........  0%  306M 22s
   250K .......... .......... .......... .......... ..........  0%  150M 18s
   300K .......... .......... .......... .......... ..........  0% 94.0M 16s
   350K .......... .......... .......... ...

In [11]:
%%bash
cat gutenberg-poetry-v001.ndjson | jq .gid | tr -d '"' > id.txt
cat gutenberg-poetry-v001.ndjson | jq .s   | sed -e 's/^.//' -e 's/.$//' -e 's/\\//g' >> poetry.txt
paste --delimiters='|' id.txt poetry.txt > gutenberg.txt
shuf gutenberg.txt | head

|With amplitude unchecked, how sweetly thou
|Of fire-waves round the infernal wall;
|Venice," "The Deluge and other Poems."  Crown 8vo.
|The barb’rous people said, of late, to mee.
5098|spider, a term signifying a naval battle, or, at least,
|In this rising generation,
1004|Before mine eyes were standing the four torches
32528|There's ne'er a darg that wonnet speed.
|Outspeeding human justice. In the end
3168|Their engines eke they reared, and with great art


In [12]:
%%bash
for filename in gutenberg-poetry-v001.txt gutenberg.txt; do
    echo "file: >> ${filename} <<"
    head "${filename}"
    echo
done

file: >> gutenberg-poetry-v001.txt <<
The Song of Hiawatha is based on the legends and stories of
many North American Indian tribes, but especially those of the
Ojibway Indians of northern Michigan, Wisconsin, and Minnesota.
They were collected by Henry Rowe Schoolcraft, the reknowned
Schoolcraft married Jane, O-bah-bahm-wawa-ge-zhe-go-qua (The
fur trader, and O-shau-gus-coday-way-qua (The Woman of the Green
Prairie), who was a daughter of Waub-o-jeeg (The White Fisher),
who was Chief of the Ojibway tribe at La Pointe, Wisconsin.
Jane and her mother are credited with having researched,
authenticated, and compiled much of the material Schoolcraft

file: >> gutenberg.txt <<
19|The Song of Hiawatha is based on the legends and stories of
19|many North American Indian tribes, but especially those of the
19|Ojibway Indians of northern Michigan, Wisconsin, and Minnesota.
19|They were collected by Henry Rowe Schoolcraft, the reknowned
19|Schoolcraft married Jane, O-bah-bahm-wawa-ge-zhe-go-qua 

check data file size

In [13]:
!find . -iname "gutenberg*.txt*" -exec du -h {} \;

233M	./gutenberg-poetry-v001.txt
281M	./gutenberg.txt


In [14]:
%tensorflow_version 1.x

TensorFlow 1.x selected.


In [15]:
!pip3 install -r requirements.txt

In [16]:
!cp src/train.py .

cp: cannot stat 'src/train.py': No such file or directory


In [17]:
#!PYTHONPATH=src python ./train.py --model_name 117M --dataset gutenberg-poetry-v001.txt.npz --batch_size 2 --save_every 10000 --sample_every 5000 --learning_rate 0.00001 --accumulate_gradients 2 

In [18]:
!ls checkpoint/run1

checkpoint  events.out.tfevents.1652443400.ff69d5f2fadb  model-3403.index
counter     model-3403.data-00000-of-00001		 model-3403.meta


In [19]:
# Save gutenberg poem ids into gberg_gids list
!cut -d '|' -f1 gutenberg.txt | sort -u > gberg_gids.list
gberg_gids = []
with open("gberg_gids.list", 'r') as f:
  for line in f.readlines():
    gberg_gids.append(line.strip())

In [20]:
gberg_gids[:10]

['', '1001', '1002', '1003', '1004', '1005', '1006', '1007', '1008', '10122']

In [21]:
def parse_input_text(input_texts):
  output_texts = []
  for line in input_texts:
    if line not in output_texts:
      output_texts.append(line)
  #import pdb; pdb.set_trace()
  return output_texts

In [22]:
from tqdm import tqdm
from lexical_diversity import lex_div as ld
import numpy as np
# load in the data
input_texts = []
songs = []
import hashlib
hashes = set([])
idx = 1
with open("/content/drive/MyDrive/t5/songs100000_2_c4.tsv", 'r') as f, open("./songs_out.txt", "w") as fw:
  for song in tqdm(f.readlines()):
    while str(idx) in gberg_gids:
      idx+=1
    input_texts = []
    if idx==0: continue
    line_ents = []
    for line in song.split('\\n'):
      line = line.rstrip()
      if not line:
        continue
      line_ents.append(len(line.split(' ')))
      input_texts.append(line)
    # sanity check:
    l = len(line_ents)
    st = np.std(line_ents)
    m = np.mean(line_ents)
    if l > 25 and st < 2 and m >= 2.75:
      filtered = parse_input_text(input_texts)
      final_chunk = '\n'.join(filtered)+'\n'
      flt = ld.flemmatize(final_chunk)
      ttr = ld.ttr(flt)
      to_write = f'{idx}|'+f'\n{idx}|'.join(filtered)+f'\n{idx}|<|endoftext|>\n'
      myhash = hashlib.sha1(to_write.encode('utf-8'))
      if ttr > 0.2 and not myhash in hashes:
         print(f"{l:.2f}, {st:.2f}, {m:.2f}, {ttr:.2f}")
         fw.write(to_write)
         hashes.add(myhash)
    idx+=1

  1%|          | 986/95379 [00:00<00:19, 4756.88it/s]

48.00, 1.22, 6.27, 0.48
36.00, 1.89, 6.44, 0.49
42.00, 1.44, 5.71, 0.50
49.00, 1.46, 5.84, 0.48
82.00, 1.82, 4.72, 0.45
97.00, 1.71, 4.53, 0.44
50.00, 1.91, 5.50, 0.56
27.00, 2.00, 5.30, 0.58
49.00, 1.33, 4.02, 0.59
36.00, 1.82, 5.31, 0.63
48.00, 1.88, 4.25, 0.61
27.00, 1.37, 4.52, 0.48
32.00, 1.74, 7.19, 0.47
33.00, 1.72, 7.21, 0.49
32.00, 1.74, 7.19, 0.47
29.00, 1.28, 5.76, 0.51
37.00, 1.20, 6.11, 0.53
37.00, 1.20, 6.11, 0.53
38.00, 1.88, 4.71, 0.64
48.00, 1.90, 6.02, 0.56
64.00, 1.92, 5.36, 0.44
26.00, 1.36, 5.42, 0.57
85.00, 1.85, 4.95, 0.49
82.00, 1.84, 4.89, 0.49
33.00, 1.23, 5.24, 0.52
62.00, 1.95, 6.71, 0.40
36.00, 1.81, 7.11, 0.56
52.00, 1.86, 7.21, 0.31
65.00, 1.64, 6.18, 0.49
34.00, 1.93, 4.56, 0.58
52.00, 1.86, 7.21, 0.31
115.00, 1.84, 6.51, 0.29
37.00, 1.88, 5.22, 0.52
40.00, 1.61, 8.05, 0.50
52.00, 1.39, 3.96, 0.48
31.00, 1.79, 7.35, 0.60
90.00, 1.98, 6.87, 0.51
41.00, 1.45, 5.73, 0.51
94.00, 1.99, 6.19, 0.36
74.00, 1.80, 4.64, 0.48
95.00, 1.69, 4.39, 0.46
33.00, 1.66, 4.

  2%|▏         | 1831/95379 [00:00<00:21, 4422.97it/s]

28.00, 1.82, 3.79, 0.68
45.00, 1.52, 4.18, 0.75
41.00, 1.31, 4.44, 0.71
38.00, 1.37, 6.39, 0.57
55.00, 1.71, 7.38, 0.44
87.00, 1.93, 4.67, 0.40
63.00, 1.96, 6.48, 0.35
34.00, 1.48, 7.41, 0.57
26.00, 1.50, 3.54, 0.42
52.00, 1.91, 4.69, 0.62
64.00, 1.78, 5.33, 0.39
39.00, 1.39, 5.49, 0.56
52.00, 1.92, 5.27, 0.48
50.00, 1.90, 5.56, 0.45
43.00, 1.76, 4.67, 0.62
64.00, 1.30, 4.28, 0.60
67.00, 1.70, 4.49, 0.52
31.00, 1.36, 4.58, 0.56
60.00, 1.97, 5.13, 0.49
60.00, 1.97, 5.13, 0.49
44.00, 1.98, 5.84, 0.43
58.00, 1.86, 4.83, 0.43
45.00, 1.13, 5.16, 0.59
42.00, 1.17, 5.10, 0.54
47.00, 1.90, 4.43, 0.64
50.00, 1.20, 3.80, 0.53
80.00, 0.96, 4.54, 0.40
38.00, 1.98, 5.13, 0.54
39.00, 1.60, 4.51, 0.60
26.00, 1.89, 6.12, 0.56
47.00, 1.74, 5.66, 0.57
34.00, 1.30, 5.65, 0.56
56.00, 1.77, 4.86, 0.46
49.00, 1.84, 5.84, 0.49
30.00, 1.94, 7.60, 0.46
37.00, 1.59, 6.03, 0.52
47.00, 1.66, 6.51, 0.42
60.00, 1.11, 4.83, 0.51
52.00, 1.82, 6.10, 0.42
28.00, 1.31, 4.82, 0.71
53.00, 1.98, 4.91, 0.73
26.00, 1.94, 5.3

  4%|▎         | 3381/95379 [00:00<00:18, 4926.53it/s]

...

38.00, 1.80, 5.53, 0.68
38.00, 1.68, 5.11, 0.67
52.00, 1.87, 5.48, 0.57
36.00, 1.51, 5.00, 0.64
35.00, 1.35, 3.69, 0.66
64.00, 1.94, 4.50, 0.57
32.00, 1.56, 3.75, 0.73
42.00, 1.90, 5.12, 0.61
46.00, 1.91, 5.39, 0.66
33.00, 1.63, 8.67, 0.60
50.00, 1.85, 4.52, 0.76
54.00, 1.75, 6.44, 0.60
40.00, 1.93, 5.90, 0.62
49.00, 1.96, 5.45, 0.66
35.00, 1.98, 5.74, 0.57
38.00, 1.04, 5.39, 0.59
30.00, 0.98, 4.03, 0.54
34.00, 1.60, 6.03, 0.45
34.00, 1.67, 4.82, 0.49
30.00, 0.60, 4.03, 0.37
31.00, 1.28, 5.03, 0.59
26.00, 1.36, 4.62, 0.57
28.00, 1.88, 5.46, 0.63
27.00, 1.41, 4.85, 0.54
35.00, 1.35, 4.66, 0.43
32.00, 0.80, 4.09, 0.70
29.00, 1.05, 6.31, 0.53
27.00, 1.91, 5.11, 0.57
32.00, 1.89, 7.19, 0.47
44.00, 1.18, 4.30, 0.55
35.00, 1.96, 6.34, 0.47
30.00, 1.35, 4.70, 0.60
26.00, 1.82, 5.00, 0.62
26.00, 1.47, 5.46, 0.73
27.00, 0.98, 5.93, 0.65
40.00, 1.69, 6.60, 0.44
36.00, 1.60, 6.64, 0.47
28.00, 1.69, 4.64, 0.62
58.00, 1.92, 5.50, 0.52
36.00, 1.54, 4.44, 0.73
38.00, 1.89, 5.79, 0.44
37.00, 1.67, 7.5

 99%|█████████▉| 94497/95379 [00:22<00:00, 3822.45it/s]

28.00, 1.58, 6.82, 0.67
30.00, 1.47, 6.37, 0.58
31.00, 1.46, 7.55, 0.55
30.00, 1.78, 4.87, 0.60
41.00, 1.87, 6.00, 0.58
63.00, 1.44, 4.51, 0.49
27.00, 1.01, 8.85, 0.57
31.00, 1.13, 6.55, 0.53
50.00, 1.85, 5.30, 0.62
26.00, 0.62, 3.38, 0.64
46.00, 1.87, 5.50, 0.54
47.00, 1.48, 4.81, 0.59
29.00, 1.35, 6.52, 0.57
65.00, 1.18, 5.72, 0.61
38.00, 1.30, 5.05, 0.45
35.00, 1.46, 7.03, 0.59
61.00, 1.79, 6.03, 0.49
33.00, 0.92, 8.58, 0.47
33.00, 1.05, 8.52, 0.49
33.00, 0.96, 8.55, 0.47
37.00, 1.78, 6.43, 0.57
39.00, 1.59, 6.33, 0.55
42.00, 1.39, 5.21, 0.61
50.00, 1.78, 7.02, 0.51
29.00, 1.95, 8.79, 0.48
48.00, 1.15, 4.52, 0.55
45.00, 1.72, 7.29, 0.51
42.00, 1.40, 6.29, 0.55
32.00, 1.36, 8.12, 0.52
38.00, 1.76, 8.11, 0.49
38.00, 1.98, 7.26, 0.56
28.00, 1.15, 6.79, 0.54
47.00, 1.91, 6.74, 0.51
39.00, 1.54, 7.72, 0.53
30.00, 1.65, 7.13, 0.54
42.00, 1.75, 6.29, 0.45
27.00, 1.05, 6.00, 0.65
31.00, 1.63, 6.74, 0.55
31.00, 1.91, 9.19, 0.60
34.00, 1.35, 5.15, 0.62
33.00, 1.56, 6.58, 0.54
30.00, 0.89, 5.9

100%|██████████| 95379/95379 [00:22<00:00, 4215.97it/s]

41.00, 1.44, 6.78, 0.62
34.00, 1.26, 6.56, 0.54
52.00, 1.87, 6.40, 0.53
44.00, 1.95, 7.73, 0.53
53.00, 1.76, 3.72, 0.54
32.00, 1.59, 6.19, 0.51
36.00, 1.61, 4.97, 0.61
30.00, 1.63, 8.47, 0.59
74.00, 1.94, 5.96, 0.43
31.00, 1.28, 5.68, 0.69
38.00, 1.50, 8.61, 0.55
71.00, 1.58, 5.42, 0.52
50.00, 1.77, 6.56, 0.56
29.00, 1.74, 6.14, 0.64
45.00, 1.75, 4.89, 0.56
52.00, 1.97, 4.40, 0.56
32.00, 0.82, 4.38, 0.39
85.00, 1.25, 4.86, 0.61
50.00, 1.69, 6.40, 0.49
28.00, 1.64, 4.46, 0.76
41.00, 1.66, 7.24, 0.56
35.00, 1.91, 4.31, 0.56
48.00, 1.72, 5.06, 0.57
41.00, 1.67, 4.78, 0.72
54.00, 1.97, 5.69, 0.50
32.00, 1.13, 6.66, 0.51
42.00, 1.01, 4.31, 0.67
96.00, 1.02, 4.33, 0.53
57.00, 1.26, 5.46, 0.47
34.00, 1.89, 5.38, 0.62
31.00, 1.54, 6.65, 0.50
35.00, 1.05, 8.46, 0.38
37.00, 1.84, 5.05, 0.59
48.00, 1.22, 5.15, 0.60
35.00, 1.00, 4.26, 0.65
48.00, 1.04, 6.04, 0.57
28.00, 1.57, 3.79, 0.65
45.00, 1.57, 4.13, 0.52
32.00, 1.84, 6.28, 0.54
30.00, 1.16, 6.67, 0.67
26.00, 1.15, 6.62, 0.53
29.00, 1.89, 8.4

In [23]:
%%bash
echo "Each of these files should have delimiting ids referring to the song/poem"
for filename in songs_out.txt gutenberg.txt; do
    echo "filename: >> ${filename}"
    printf "lines: "
    wc -l "${filename}"
    printf "size: "
    du -h "${filename}"
    printf "ids: "
    cut -d '|' -f1 "${filename}" | sort -u | wc -l | tr -d '\n' && echo "    ${filename}"
    echo
done

Each of these files should have delimiting ids referring to the song/poem
filename: >> songs_out.txt
lines: 470879 songs_out.txt
size: 16M	songs_out.txt
ids: 14585    songs_out.txt

filename: >> gutenberg.txt
lines: 6901560 gutenberg.txt
size: 281M	gutenberg.txt
ids: 1192    gutenberg.txt



In [24]:
!head gutenberg.txt

19|The Song of Hiawatha is based on the legends and stories of
19|many North American Indian tribes, but especially those of the
19|Ojibway Indians of northern Michigan, Wisconsin, and Minnesota.
19|They were collected by Henry Rowe Schoolcraft, the reknowned
19|Schoolcraft married Jane, O-bah-bahm-wawa-ge-zhe-go-qua (The
19|fur trader, and O-shau-gus-coday-way-qua (The Woman of the Green
19|Prairie), who was a daughter of Waub-o-jeeg (The White Fisher),
19|who was Chief of the Ojibway tribe at La Pointe, Wisconsin.
19|Jane and her mother are credited with having researched,
19|authenticated, and compiled much of the material Schoolcraft


In [25]:
!head ./songs_out.txt

2|Give you what you want
2|Give you what you need
2|Baby, give you what you need
2|I know it gets hard sometimes
2|When you feel that you'll never find
2|That one who will always be there
2|When you need him there, that's all I want
2|All I want is someone nice
2|To make me feel like paradise
2|All I need is someone real


In [26]:
!cat ./songs_out.txt ./gutenberg.txt > ./songs-out_gutenberg-poetry-v001.txt

In [27]:
!du -h ./songs_out*.txt ./gutenberg*.txt  ./songs-gutenberg-poetry-v001.txt

16M	./songs_out.txt
233M	./gutenberg-poetry-v001.txt
281M	./gutenberg.txt
183M	./songs-gutenberg-poetry-v001.txt


In [28]:
!head ./songs-out_gutenberg-poetry-v001.txt

2|Give you what you want
2|Give you what you need
2|Baby, give you what you need
2|I know it gets hard sometimes
2|When you feel that you'll never find
2|That one who will always be there
2|When you need him there, that's all I want
2|All I want is someone nice
2|To make me feel like paradise
2|All I need is someone real


In [29]:
!grep -Pv "^\|.*" ./songs-out_gutenberg-poetry-v001.txt > ./songs-out_gutenberg-poetry-v001_fixed.txt

In [30]:
%%bash
for filename in ./songs-out_gutenberg-poetry-v001_fixed.txt ; do
    echo "filename: >> ${filename}"
    printf "lines: "
    wc -l "${filename}"
    printf "size: "
    du -h "${filename}"
    printf "ids: "
    cut -d '|' -f1 "${filename}" | sort -u | wc -l | tr -d '\n' && echo "    ${filename}"
    echo
done

filename: >> ./songs-out_gutenberg-poetry-v001_fixed.txt
lines: 3555996 ./songs-out_gutenberg-poetry-v001_fixed.txt
size: 149M	./songs-out_gutenberg-poetry-v001_fixed.txt
ids: 15776    ./songs-out_gutenberg-poetry-v001_fixed.txt



Extract GWERN's pretrained poetry model

In [31]:
#with open(", 'r', encoding="ISO-8859-1") as f:
#  with open("./models/2019-03-06-gwern-gpt2-poetry-prefix-projectgutenberg-network-224474.tar.xz", "w") as fw:
#    fw.write(f.read())
#import tarfile
import tarfile
with tarfile.open("/content/drive/MyDrive/t5/2019-03-06-gwern-gpt2-poetry-prefix-projectgutenberg-network-224474.tar.xz") as f:
    f.extractall('.')

encode our data

In [32]:
%%bash
[ -f "models/117M/encoder.json" ] || cp models/2019-03-06-gwern-gpt2-poetry-prefix-projectgutenberg-network-224474/encoder.json models/117M/encoder.json && echo "copied encoder to model/117M"
[ -f "models/117M/vocab.bpe" ] || cp models/2019-03-06-gwern-gpt2-poetry-prefix-projectgutenberg-network-224474/vocab.bpe models/117M/vocab.bpe && echo "copied bpe vocab to model/117M"

copied encoder to model/117M
copied bpe vocab to model/117M


In [33]:
#!PYTHONPATH=src python ./encode.py ./songs-out_gutenberg-poetry-v001_fixed.txt{,.npz}

Reading files
  0% 0/1 [00:00<?, ?it/s]^C


In [ ]:
!PYTHONPATH=src python ./encode.py ./songs_out.txt{,.npz}

In [34]:
!du -h ./songs-out_gutenberg-poetry-v001_fixed.txt*

149M	./songs-out_gutenberg-poetry-v001_fixed.txt
67M	./songs-out_gutenberg-poetry-v001_fixed.txt.npz


In [35]:
!ls 2019-03-06-gwern-gpt2-poetry-prefix-projectgutenberg-network-224474

checkpoint  model-224475.data-00000-of-00001  model-224475.meta
counter     model-224475.index


In [36]:
%%bash
echo "copy data to gwern model in order to train"
gwern_poetry_prefix="2019-03-06-gwern-gpt2-poetry-prefix-projectgutenberg-network-224474"
cp -v ./models/117M/encoder.json "$gwern_poetry_prefix"
cp -v ./models/117M/vocab.bpe "$gwern_poetry_prefix"
cp -v ./models/117M/hparams.json "$gwern_poetry_prefix"
echo
echo "moving $gwern_poetry_prefix to models/"
mv -v "$gwern_poetry_prefix" models/

copy data to gwern model in order to train
'./models/117M/encoder.json' -> '2019-03-06-gwern-gpt2-poetry-prefix-projectgutenberg-network-224474/encoder.json'
'./models/117M/vocab.bpe' -> '2019-03-06-gwern-gpt2-poetry-prefix-projectgutenberg-network-224474/vocab.bpe'
'./models/117M/hparams.json' -> '2019-03-06-gwern-gpt2-poetry-prefix-projectgutenberg-network-224474/hparams.json'

moving 2019-03-06-gwern-gpt2-poetry-prefix-projectgutenberg-network-224474 to models/


mv: cannot move '2019-03-06-gwern-gpt2-poetry-prefix-projectgutenberg-network-224474' to 'models/2019-03-06-gwern-gpt2-poetry-prefix-projectgutenberg-network-224474': Directory not empty


In [37]:
!cp 2019-03-06-gwern-gpt2-poetry-prefix-projectgutenberg-network-224474/* ./models/2019-03-06-gwern-gpt2-poetry-prefix-projectgutenberg-network-224474/

In [38]:
!ls models/2019-03-06-gwern-gpt2-poetry-prefix-projectgutenberg-network-224474/

2019-03-06-gwern-gpt2-poetry-prefix-projectgutenberg-network-224474
checkpoint
counter
encoder.json
hparams.json
model-224475.data-00000-of-00001
model-224475.index
model-224475.meta
vocab.bpe


In [39]:
!cat models/2019-03-06-gwern-gpt2-poetry-prefix-projectgutenberg-network-224474/counter
!cat models/2019-03-06-gwern-gpt2-poetry-prefix-projectgutenberg-network-224474/hparams.json
!cat models/2019-03-06-gwern-gpt2-poetry-prefix-projectgutenberg-network-224474/checkpoint

224475
{
  "n_vocab": 50257,
  "n_ctx": 1024,
  "n_embd": 768,
  "n_head": 12,
  "n_layer": 12
}
model_checkpoint_path: "model-224475"
all_model_checkpoint_paths: "model-200000"
all_model_checkpoint_paths: "model-220000"
all_model_checkpoint_paths: "model-224475"


In [ ]:
!PYTHONPATH=src python3 train.py --model_name 2019-03-06-gwern-gpt2-poetry-prefix-projectgutenberg-network-224474 --restore_from fresh --dataset ./songs_out.txt --batch_size 5 --save_every 5000 --sample_every 1000 --sample_length 1000

2022-05-13 19:16:29.398155: E tensorflow/stream_executor/cuda/cuda_driver.cc:271] failed call to cuInit: CUDA_ERROR_NO_DEVICE: no CUDA-capable device is detected
Instructions for updating:
Use `tf.cast` instead.
Instructions for updating:
Use `tf.random.categorical` instead.
Using Adam optimizer
Loading checkpoint models/2019-03-06-gwern-gpt2-poetry-prefix-projectgutenberg-network-224474/model-224475
Loading dataset...
100% 1/1 [00:20<00:00, 20.77s/it]
dataset has 5284768 tokens
Training...
[3404 | 15.69] loss=3.76 avg=3.76
[3405 | 27.24] loss=4.11 avg=3.93
[3406 | 38.36] loss=3.32 avg=3.73
[3407 | 49.59] loss=3.69 avg=3.72
[3408 | 61.02] loss=4.05 avg=3.78
[3409 | 72.55] loss=3.52 avg=3.74
[3410 | 83.81] loss=3.90 avg=3.76
[3411 | 95.09] loss=3.62 avg=3.74
[3412 | 106.35] loss=3.30 avg=3.69
[3413 | 117.70] loss=3.76 avg=3.70
[3414 | 129.49] loss=3.84 avg=3.71
[3415 | 140.74] loss=3.63 avg=3.71
[3416 | 151.79] loss=3.43 avg=3.68
[3417 | 162.88] loss=3.70 avg=3.69
[3418 | 173.62] loss=3

In [ ]:
!ls checkpoint/run1/

In [ ]:
!pip install tensorboard

In [ ]:
!ls models/*/*

In [ ]:
!cp -r models/2019-03-06-gwern-gpt2-poetry-prefix-projectgutenberg-network-224474/ models/2019-03-06-gwern-gpt2-poetry-prefix-projectgutenberg-network-224474-3403

In [ ]:
%%bash
gwern_poetry_prefix=models/2019-03-06-gwern-gpt2-poetry-prefix-projectgutenberg-network-224474
echo "${gwern_poetry_prefix}-3403"
cp -v $gwern_poetry_prefix/encoder.json "${gwern_poetry_prefix}-3403"
cp -v $gwern_poetry_prefix/vocab.bpe "${gwern_poetry_prefix}-3403"
cp -v $gwern_poetry_prefix/hparams.json "${gwern_poetry_prefix}-3403"
cp -v checkpoint/run1/model-3403.data-00000-of-00001 "${gwern_poetry_prefix}-3403"

In [ ]:
!du -h checkpoint/run1/model-3403.data-00000-of-00001

In [ ]:
!du -h models/2019-03-06-gwern-gpt2-poetry-prefix-projectgutenberg-network-224474/model-224475.data-00000-of-00001

In [ ]:
!ls models/2019-03-06-gwern-gpt2-poetry-prefix-projectgutenberg-network-224474/2019-03-06-gwern-gpt2-poetry-prefix-projectgutenberg-network-224474

In [ ]:
%tensorflow_version 2.x

In [ ]:
%load_ext tensorboard

In [ ]:
%tensorboard --logdir checkpoint/run1

In [ ]:
! [ ! -f "2019-03-06-gwern-gpt2-poetry-prefix-projectgutenberg-network-224474" ] || mv 2019-03-06-gwern-gpt2-poetry-prefix-projectgutenberg-network-224474 models/
#!mv models/117M/*{json,bpe} ./models/2019-03-06-gwern-gpt2-poetry-prefix-projectgutenberg-network-224474/
!python3 src/generate_unconditional_samples.py --top_k 40 --temperature 0.9 --nsamples 10000 --seed 0 --model_name 2019-03-06-gwern-gpt2-poetry-prefix-projectgutenberg-network-224474-3403

In [ ]:
!python3 src/interactive_conditional_samples.py --top_k 40 --temperature 0.9 --nsamples 10000 --seed 0 --model_name 2019-03-06-gwern-gpt2-poetry-prefix-projectgutenberg-network-224474-3403

In [ ]:
from tqdm import tqdm
from collections import Counter
with open("/content/drive/MyDrive/t5/songs100000_2_c4.tsv", 'r') as f:
  for idx, song in tqdm(enumerate(f.readlines(), 0)):
    input_texts = []
    if idx==0: continue
    for line in song.split('\\n'):
      line = line.rstrip()
      if not line:
        continue
      input_texts.append(line)
    #input_texts.append()
    fileout = '\n'.join(input_texts) + '<|endoftext|>\n'
    import pdb; pdb.set_trace()
    #fw.write(final_chunk)

In [ ]:
!PYTHONPATH=src python3 train.py --model_name 2019-03-06-gwern-gpt2-poetry-prefix-projectgutenberg-network-224474 --dataset ./songs-gutenberg-poetry-v002.txt.npz --batch_size 2 --save_every 10000 --sample_every 1000 --sample_length 1000

In [ ]:
#!ls models/2019-03-06-gwern-gpt2-poetry-prefix-projectgutenberg-network-224474

In [ ]:
ls models/117MR

In [ ]:
!rm models/117MR/model.ckpt.data-00000-of-00001 models/117MR/model.ckpt.index models/117MR/model.ckpt.meta

In [ ]:
!mv models/117MR/model-2764.data-00000-of-00001 models/117MR/model.ckpt.data-00000-of-00001
!mv models/117MR/model-2764.index models/117MR/model.ckpt.index
!mv models/117MR/model-2764.meta models/117MR/model.ckpt.meta

In [ ]:
!ls models/117M/*{json,bpe} ./models

In [ ]:
!cp -r models/117M models/117MR

In [ ]:
!ls models/117M

In [ ]:
!ls models/117M*

In [ ]:
!cp checkpoint/run1/model-2764.meta models/117MR/
!cp checkpoint/run1/model-2764.data-00000-of-00001 models/117MR/
!cp checkpoint/run1/model-2764.index models/117MR/

In [ ]:
!ls checkpoint/run1/*

In [ ]:
!ls src

In [ ]:
!ls nsheppy

In [ ]:
!grep -P "^523\|" ./songs_out.txt

In [ ]:
!ls

In [ ]:
# If you are using a bigger Drive (e.g. by paying a little for Google One) just copy the whole project there once and work from Drive
# Otherwise copy just the model weights in and out of Drive and keep a single copy of them.

!cp -r gpt-2/ /content/drive/My\ Drive/gpt-2/gpt-2/

In [ ]:
%cd /content/drive/My\ Drive/gpt-2/gpt-2/

In [ ]:
!PYTHONPATH=src python3 train.py  --model_name 177M --dataset songs_out.txt --batch_size 10 --save_every -1000 --sample_every 100 --sample_num 1 --sample_length 190  --save_on_ctrlc --save_time 100 --max_to_keep 1 --learning_rate 0.00001   --accumulate_gradients 2 --optimizer sgd #--memory_saving_gradients 


In [ ]:
!PYTHONPATH=src python3 train.py  --model_name 1558M --dataset fb-cleaned.txt.npz --batch_size 10 --save_every -1 --sample_every 190 --sample_num 1 --sample_length 190  --save_on_ctrlc --save_time 119 --max_to_keep 1 --learning_rate 0.000005   --only_train_transformer_layers 
